Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [44]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.365522
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.800873
Minibatch accuracy: 43.8%
Validation accuracy: 49.2%
Minibatch loss at step 100: 1.119838
Minibatch accuracy: 56.2%
Validation accuracy: 65.0%
Minibatch loss at step 150: 0.675118
Minibatch accuracy: 75.0%
Validation accuracy: 75.1%
Minibatch loss at step 200: 0.894705
Minibatch accuracy: 68.8%
Validation accuracy: 74.9%
Minibatch loss at step 250: 1.199480
Minibatch accuracy: 62.5%
Validation accuracy: 76.6%
Minibatch loss at step 300: 0.626384
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 350: 0.541163
Minibatch accuracy: 81.2%
Validation accuracy: 79.7%
Minibatch loss at step 400: 0.632871
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 450: 0.994450
Minibatch accuracy: 68.8%
Validation accuracy: 79.7%
Minibatch loss at step 500: 0.317827
Minibatch accuracy: 93.8%
Validation accuracy: 80.8%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [8]:
batch_size = 16
#patch_size = 5
patch_size = 6
#depth = 16
depth = 32
num_hidden = 64

beta = 0.01

dropout = 0.5

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    
#    hidden_drop = tf.nn.dropout(hidden, dropout)
    
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
#    hidden_drop = tf.nn.dropout(hidden, dropout)
    
    
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')
    
    shape = pool.get_shape().as_list()
    
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+
    beta*tf.nn.l2_loss(layer3_weights) +
    beta*tf.nn.l2_loss(layer3_biases) +
    beta*tf.nn.l2_loss(layer4_weights) +
    beta*tf.nn.l2_loss(layer4_biases)

    )
    
  # Optimizer.

  global_step = tf.Variable(0)  # count the number of steps taken.
#  learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.96,staircase=True)
    
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)


  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 110001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob: dropout}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 10000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 9.145397
Minibatch accuracy: 12.5%
Validation accuracy: 11.7%
Minibatch loss at step 10000: 1.218451
Minibatch accuracy: 81.2%
Validation accuracy: 87.9%
Minibatch loss at step 20000: 0.625641
Minibatch accuracy: 81.2%
Validation accuracy: 89.1%
Minibatch loss at step 30000: 0.247096
Minibatch accuracy: 93.8%
Validation accuracy: 89.7%
Minibatch loss at step 40000: 0.803514
Minibatch accuracy: 87.5%
Validation accuracy: 90.2%
Minibatch loss at step 50000: 0.789855
Minibatch accuracy: 81.2%
Validation accuracy: 89.8%
Minibatch loss at step 60000: 0.310154
Minibatch accuracy: 87.5%
Validation accuracy: 90.2%
Minibatch loss at step 70000: 0.383093
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Minibatch loss at step 80000: 0.544057
Minibatch accuracy: 87.5%
Validation accuracy: 90.6%
Minibatch loss at step 90000: 0.156859
Minibatch accuracy: 93.8%
Validation accuracy: 90.6%
Minibatch loss at step 100000: 0.094230
Minibatch accuracy: 100.0%
Valid

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [5]:
import time
import hashlib

t1 = time.time()

train_hashes = [hashlib.sha1(x).digest() for x in train_dataset]
valid_hashes = [hashlib.sha1(x).digest() for x in valid_dataset]
test_hashes  = [hashlib.sha1(x).digest() for x in test_dataset]

valid_in_train = np.in1d(valid_hashes, train_hashes)
test_in_train  = np.in1d(test_hashes,  train_hashes)
test_in_valid  = np.in1d(test_hashes,  valid_hashes)

valid_keep = ~valid_in_train
test_keep  = ~(test_in_train | test_in_valid)
    
valid_dataset_clean = valid_dataset[valid_keep]
valid_labels_clean  = valid_labels [valid_keep]

test_dataset_clean = test_dataset[test_keep]
test_labels_clean  = test_labels [test_keep]

t2 = time.time()

print("Time: %0.2fs" % (t2 - t1))
print("valid -> train overlap: %d samples" % valid_in_train.sum())
print("test  -> train overlap: %d samples" % test_in_train.sum())
print("test  -> valid overlap: %d samples" % test_in_valid.sum())


k1= np.array([1,2,3,4])
k2 = np.array([3,5,6])
maybe_same = np.in1d(k1,k2)
print(maybe_same)

Time: 1.10s
valid -> train overlap: 1098 samples
test  -> train overlap: 1296 samples
test  -> valid overlap: 214 samples
[False False  True False]


In [6]:
pickle_file = 'notMNIST_clean.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset_clean': valid_dataset_clean,
    'valid_labels_clean': valid_labels_clean,
    'test_dataset_clean': test_dataset_clean,
    'test_labels_clean': test_labels_clean,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [7]:
batch_size = 16
#patch_size = 5
patch_size = 6
#depth = 16
depth = 32
num_hidden = 2048
num_hidden2 = 1024

beta = 0.01

dropout = 0.5

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset_clean)
  tf_test_dataset = tf.constant(test_dataset_clean)
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
    
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    
  add3_weights = tf.Variable(tf.truncated_normal(
      [num_hidden,num_hidden2], stddev=0.1))
  add3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))  
       
  
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    
#    hidden_drop = tf.nn.dropout(hidden, dropout)
    
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
#    hidden_drop = tf.nn.dropout(hidden, dropout)
    
    
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')
    
        
    shape = pool.get_shape().as_list()
    
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden_drop = tf.nn.dropout(hidden, dropout)
    
    hidden2 = tf.nn.relu(tf.matmul(hidden_drop, add3_weights) + add3_biases)
    
    return tf.matmul(hidden2, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+
    beta*tf.nn.l2_loss(layer3_weights) +
    beta*tf.nn.l2_loss(layer3_biases) +
     beta*tf.nn.l2_loss(add3_weights) + 
     beta*tf.nn.l2_loss(add3_biases) +
    beta*tf.nn.l2_loss(layer4_weights) +
    beta*tf.nn.l2_loss(layer4_biases)

    )
    
  # Optimizer.

  global_step = tf.Variable(0)  # count the number of steps taken.
#  learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
  learning_rate = tf.train.exponential_decay(0.01, global_step, 10000, 0.95,staircase=True)
    
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

#  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
#  optimizer = tf.train.MomentumOptimizer(0.1,0.9).minimize(loss, global_step=global_step)
#  optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 3000001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob: dropout}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 10000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels_clean))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels_clean))

Initialized
Minibatch loss at step 0: 272.922119
Minibatch accuracy: 6.2%
Validation accuracy: 11.3%
Minibatch loss at step 10000: 30.654238
Minibatch accuracy: 81.2%
Validation accuracy: 85.6%
Minibatch loss at step 20000: 5.166627
Minibatch accuracy: 75.0%
Validation accuracy: 87.3%
Minibatch loss at step 30000: 0.969677
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 40000: 1.048527
Minibatch accuracy: 81.2%
Validation accuracy: 88.1%
Minibatch loss at step 50000: 0.986067
Minibatch accuracy: 75.0%
Validation accuracy: 88.4%
Minibatch loss at step 60000: 0.389468
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 70000: 0.404511
Minibatch accuracy: 93.8%
Validation accuracy: 88.6%
Minibatch loss at step 80000: 0.525409
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 90000: 0.189812
Minibatch accuracy: 100.0%
Validation accuracy: 89.4%
Minibatch loss at step 100000: 0.145842
Minibatch accuracy: 100.0%
Va